# Patients exploration

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import utils as mutil
import sklearn
import sklearn.ensemble
import sklearn.model_selection

from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

## Load dataset

In [39]:
patients_ds = pd.read_csv("patient/dataset.csv")
patients_ds = patients_ds.drop(columns=["Unnamed: 83"])

## Layouttitanic_train.head()

In [40]:
patients_ds.head()


,encounter_id,patient_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,icu_admit_source,...,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem,hospital_death
0,66154,25312,118,68.0,22.73,0,Caucasian,M,180.3,Floor,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular,0
1,114252,59342,81,77.0,27.42,0,Caucasian,F,160.0,Floor,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Respiratory,Respiratory,0
2,119783,50777,118,25.0,31.95,0,Caucasian,F,172.7,Accident & Emergency,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Metabolic,Metabolic,0
3,79267,46918,118,81.0,22.64,1,Caucasian,F,165.1,Operating Room / Recovery,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Cardiovascular,Cardiovascular,0
4,92056,34377,33,19.0,NaN,0,Caucasian,M,188.0,Accident & Emergency,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Trauma,Trauma,0


In [5]:
patients_ds.describe(include='all')

,encounter_id,patient_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,icu_admit_source,...,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem,Unnamed: 83,hospital_death
count,91713.000000,91713.000000,91713.000000,87485.000000,88284.000000,91713.000000,90318,91688,90379.000000,91601,...,90998.000000,90998.000000,90998.000000,90998.000000,90998.000000,90998.000000,90051,90051,0.0,91713.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,6,2,NaN,5,...,NaN,NaN,NaN,NaN,NaN,NaN,11,10,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,Caucasian,M,NaN,Accident & Emergency,...,NaN,NaN,NaN,NaN,NaN,NaN,Cardiovascular,Cardiovascular,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,70684,49469,NaN,54060,...,NaN,NaN,NaN,NaN,NaN,NaN,29999,38816,NaN,NaN
mean,65606.079280,65537.131464,105.669262,62.309516,29.185818,0.183736,NaN,NaN,169.641588,NaN,...,0.225192,0.012989,0.026165,0.007066,0.004132,0.020638,NaN,NaN,NaN,0.086302
std,37795.088538,37811.252183,62.854406,16.775119,8.275142,0.387271,NaN,NaN,10.795378,NaN,...,0.417711,0.113229,0.159628,0.083763,0.064148,0.142169,NaN,NaN,NaN,0.280811
min,1.000000,1.000000,2.000000,16.000000,14.844926,0.000000,NaN,NaN,137.200000,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.000000
25%,32852.000000,32830.000000,47.000000,52.000000,23.641975,0.000000,NaN,NaN,162.500000,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.000000
50%,65665.000000,65413.000000,109.000000,65.000000,27.654655,0.000000,NaN,NaN,170.100000,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.000000
75%,98342.000000,98298.000000,161.000000,75.000000,32.930206,0.000000,NaN,NaN,177.800000,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.000000


In [41]:
# print("Columns:", patients_ds.columns.values)

# Convert Categorical to numeric
for t, c in zip(patients_ds.dtypes, patients_ds.columns.values):
    if t == 'object':
        print("\x1b[31m", t, "\t", c, "\x1b[0m")
        patients_ds[c] = pd.Categorical(patients_ds[c])
        patients_ds[c] = patients_ds[c].cat.codes

# Convert problematic types
float64_cols = list(patients_ds.select_dtypes(include='float64'))
patients_ds[float64_cols] = patients_ds[float64_cols].astype('float32')

int64_cols = list(patients_ds.select_dtypes(include='int64'))
patients_ds[int64_cols] = patients_ds[int64_cols].astype('int32')

# Print the stuff
import tabulate
data = [["Column", "dtype", "# NaN"]]
for t, c in zip(patients_ds.dtypes, patients_ds.columns.values):
    if t == 'object':
        print("\x1b[31m", t, "\t", c, "\x1b[0m")
    # print(t, "\t", c)
    data.append([c, t, patients_ds[c].isna().sum()])
        
table = tabulate.tabulate(data, tablefmt='html')
table

 object 	 ethnicity 
 object 	 gender 
 object 	 icu_admit_source 
 object 	 icu_stay_type 
 object 	 icu_type 
 object 	 apache_3j_bodysystem 
 object 	 apache_2_bodysystem 


Column,dtype,# NaN
encounter_id,int32,0
patient_id,int32,0
hospital_id,int32,0
age,float32,4228
bmi,float32,3429
elective_surgery,int32,0
ethnicity,int8,0
gender,int8,0
height,float32,1334
icu_admit_source,int8,0


# Notes:
* maybe to complex, switch to https://www.kaggle.com/datasets/saurabhshahane/in-hospital-mortality-prediction ???

## Train, Test, Validation split

In [35]:
train, val, test = mutil.split_train_val_test(patients_ds, validation_size=0.1, test_size=0.2)

In [36]:
# No missing labels
train['hospital_death'].isna().sum()

0

In [37]:
# Create training dataset
train_x = train.drop(columns=['hospital_death'])
train_y = train['hospital_death']
# Train model based on best estimated configuration from previous tests
model = sklearn.ensemble.RandomForestClassifier(min_samples_split= 10, n_estimators= 50, criterion='gini', min_samples_leaf= 1, class_weight="balanced")
model = model.fit(X=train_x, y=train_y)
# Calculate train accuracy
train_accuracy = model.score(X=train_x, y=train_y)
print(f"Train accuracy {train_accuracy}")
print(f"Training set utilisation = {100 * len(titanic_train_clean) / len(titanic_train)}%")

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').